# GPU-Accelerated Data Science in Python

Workshop exploring methods for GPU-accelerated data science in the Python programming language.

Requirements:
- NVIDIA GPU
- Container runtime for [nvcr.io/nvidia/pytorch:25.03-py3](https://catalog.ngc.nvidia.com/orgs/nvidia/containers/pytorch) or [nvcr.io/nvidia/rapidsai/base:25.04-cuda12.8-py3.11](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/rapidsai/containers/base/tags)
    - with [jupyterlab_nvdashboard](https://github.com/rapidsai/jupyterlab-nvdashboard)
- Experience with Python
 
Outcomes:
- Explore GPU-accelerated Data Science Packages
- Understand when GPU acceleration makes sense
- Learn how to work with data that doesn't fit in GPU memory
- Learn how to across multiple GPUs

## Introduction
<hr>

GPU-accelerated data science involves using GPUs to perform data-intensive computations more quickly than traditional Central Processing Units (CPUs).
GPUs excel in tasks that involve large-scale parallel processing, making them ideal for machine learning, deep learning, data processing, and scientific computing.

This workshop is meant to be generally applicable who already use python for data science.
We'll be learning how to adapt CPU-based data-science methods to run on the GPU.

## Quickstart with K-means on the GPU
<hr>

If you only have a few minutes, this section will get you started with GPU-accelerated data science.
We'll be using the k-means algorithm as our example algorithm because the methods are relatively simple and it has been re-implemented in many different python packages.

If you've never use the k-means algorith before, its goal is to partition data into `k` clusters by minimizing the variance within each cluster.
The algorithm starts by:

1. Initializing `k` centroids randomly (usually by choosing random data points without replacement)
1. Each data point is then assigned to the nearest centroid
1. The centroids are updated to be the mean of the assigned points
1. This process is repeated until convergence, where assignments no longer change.

If you're interested in learning more, check out the [wikipedia page](https://en.wikipedia.org/wiki/K-means_clustering).

### Starting from scratch

While most folks usually start from a pre-implemented version of k-means, lets say you wrote a python script like [kmeans.py](kmeans.py). This script implements the k-means algorithm using array based operations in [numpy](https://numpy.org/).

Take a look at the script to see what it's doing, but no need to master the content.

You'll notice that it focuses on minimizing `for` loops by using numpy's array-based operations.
This is necessary for performance in python since loops cannot be vectorized.

Since the script is in the same directory as this notebook, we can import it and run it as follows:

In [ ]:
import os
os.environ['OMP_NUM_THREADS']=str(min(16,os.cpu_count())) # Necessary on nodes with many cores

from kmeans import k_means # Load our numpy implementation of k-means
import numpy as np # Use numpy to create and work with data
import matplotlib.pyplot as plt # We'll plot the data
from time import time # Allows us to time blocks of code

# Create some data
data = np.array([[1, 2], [1.5, 1.8], [5, 8], [8, 8], [1, 0.6], [99, 100]])
k = 3  # Number of clusters

# Run K-means
assignments, centroids = k_means(data, k)

# Print the results (you can modify this for your needs)
print("Final Centroids:\n", centroids)
print("Cluster Assignments:", assignments)

### Generating large data

This numpy implementation works great for our current data.
What happens if data grows though?

To answer this question, we can create a function to generate large quantities of clustered data to see how this runs as data scales.

In [ ]:
def generate_data(k=3, n=10, d=2, cmin=0, cmax=10):
    '''
    Generate a random dataset for clustering.
    k - number of clusters
    n - number of data points
    d - dimension of data points
    cmin - smallest centroid value
    cmax - largest centroid value
    '''
    # Generate random integers for centroids
    C = np.random.randint(cmin, cmax, size=(k,d))
    # Create data points by adding random noise to centroids
    D = np.random.randn(n, d)+C[np.random.choice(k, n, replace=True)]
    return D, C

data, centroids = generate_data(k=3, n=15, d=2)
print("Centroids:\n",centroids,"\n")
print("Data Points:\n",data)

### Plotting 2D clusters

To help understand the data that is generated, we can visualize it.
So we don't have to cover how to use [Matplotlib](https://matplotlib.org/) in this workshop, the [extras.py](extras.py) script contains a `plot_clusters` function for plotting 2D data.

Feel free to regernate data if the clusters don't make sense.

In [ ]:
from extras import plot_clusters

plot_clusters(data, actual=centroids)

### Running K-means

Try running the numpy implementation of K-means using the `k_means` function and visualize the cluster assignments using `plot_clusters` again.

In [ ]:
assignments, calc_centroids = k_means(data, k=3)
print(calc_centroids)

In [ ]:
plot_clusters(data, assignments, k=3, calculated=calc_centroids, actual=centroids)

This is mostly to prove we can now generate clusters of data in any shape.
I'm focusing on the centroids here so we don't have to get into assignment accuracy.
As the number of data points increases, the calculated centroids should get closer to actual centroids used to generate the data.
Feel free to take some time to try increasing `n` when `generate_data` is called to verify this.

In [ ]:
# Experiment with different counts and dimensions of data

data, centroids = generate_data(k=3, n=100, d=2)
plot_clusters(data, actual=centroids)

### Timing numpy implementation

Now, lets look at runtime as the data grows from 10 to 10,000,000 (10^7) data points.

Notice that we're only running K-means for a single iteration with `max_iterations=1`.
This is because the k-means algorithm is an NP-hard algorithm with a non-deterministic runtime (all the random selections) because it loops until convergence.
To give us an idea of how fast each loop of the algorithm runs, we're just doing a single iteration.

In [ ]:
k, d = 4, 10

for N in [10**power for power in range(1,8)]:
    data, real_centroids = generate_data(k=k, n=N, d=d, cmin=0, cmax=100)
    start_time = time()
    # Time a single iteration because k-means is non-deterministic
    k_means(data, k, max_iterations=1)
    elapsed_time = time()-start_time
    print(f"Finished kmeans on {N} points in {np.round(elapsed_time,4)} seconds")
    del data, real_centroids
print("Done") # I sometimes wait for another step

### Timing Scikit-learn implementation

You may already know this, but the k-means algorithm and may others already exist in a package called [scikit-learn](https://scikit-learn.org/stable/), an open-source, widely-used Python package for machine learning. It provides a comprehensive set of algorithms for classification, regression, clustering, dimensionality reduction, and more, along with tools for model selection, data preprocessing, and visualization.

<img src="https://raw.githubusercontent.com/scikit-learn/scikit-learn/refs/heads/main/doc/logos/scikit-learn-logo.png" alt="scikit" style="margin: 0 auto;" />

The `KMeans` class exists in the "cluster" collection.
Take a look at the documentation to learn how to use it.

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

In [ ]:
import os
os.environ['OMP_NUM_THREADS']=str(min(16,os.cpu_count())) # Necessary on nodes with many cores
from sklearn.cluster import KMeans

k, d = 4, 10

for N in [10**power for power in range(1,8)]:
    data, real_centroids = generate_data(k=k, n=N, d=d, cmin=0, cmax=100)
    start_time = time()
    assignments = KMeans(n_clusters=k, random_state=0, max_iter=1).fit_predict(data)
    elapsed_time = time()-start_time
    print(f"Finished sklearn-kmeans on {N} points in {np.round(elapsed_time,4)} seconds")
    del data, real_centroids
print("Done")

You'll notice that for small cases, our numpy implementation is faster.
However, as data grows, the scikit-learn implmentation becomes faster.
If you open up a view of the CPU utilization, you'll see that scikit-learn algorithms can use multiple CPU cores.

#### Exercise:
- Use `top` to look at CPU utilization while running KMeans

### Timing cuML Implementation

We've spent a lot of time on CPU code, but this workshop is meant to showcase GPU acceleration.
One of the ways we'll explore GPU acceleration is through the RAPIDS ecosystem.

<img src="https://raw.githubusercontent.com/rapidsai/rapids.ai/refs/heads/main/assets/images/RAPIDS-logo.png" alt="rapids" style="margin: 0 auto; width: 50%;" />
<br>

[RAPIDS](https://rapids.ai) provides unmatched speed with familiar APIs that match the most popular PyData libraries. Built on state-of-the-art foundations like NVIDIA CUDA and Apache Arrow, it unlocks the speed of GPUs with code you already know.

<html>
<table><thead>
  <tr>
    <th></th>
    <th><b>CPU</b></th>
    <th bgcolor="#7400ff" style="color:white;"><b>RAPIDS - GPU</b></th>
  </tr></thead>
<tbody>
  <tr>
    <td>Pandas</td>
    <td><pre>import pandas as pd<br>df = pd.read_csv("filepath")</pre></td>
    <td><pre>import cudf<br>df = cudf.read_csv("filepath")</pre></td>
  </tr>
  <tr>
    <td>Spark</td>
    <td><pre>spark.sql("""<br>select<br>    order<br>    count(*) as order_count<br>from<br>    orders""")</pre></td>
    <td><pre>spark.conf.set("spark.plugins",<br>"com.nvidia.spark.SQLPlugin")<br>spark.sql("""<br>select<br>    order<br>    count(*) as order_count<br>from<br>    orders""")</pre></td>
  </tr>
  <tr>
    <td>scikit-learn</td>
    <td><pre>from sklearn.ensemble import RandomForestClassifier<br>clf = RandomForestClassifier()<br>clf.fit(x,y)</pre></td>
    <td><pre>from cuml.ensemble import RandomForestClassifier<br>clf = RandomForestClassifier()<br>clf.fit(x,y)</pre></td>
  </tr>
  <tr>
    <td>NetworkX</td>
    <td><pre>import networkx as nx<br>page_rank = nx.pagerank(graph)</pre></td>
    <td><pre>import cugraph<br>page_rank = cugraph.pagerank(graph)</pre></td>
  </tr>
</tbody></table>
</html>

Based on convention, there should be a [cuml.cluster.KMeans](https://docs.rapids.ai/api/cuml/stable/api/#k-means-clustering) that matches the `sklearn.cluster.KMeans` we previously used.

> The documentation also states that this function can be imported with either `cuml.KMeans` and `cuml.cluster.KMeans`

In [ ]:
from cuml.cluster import KMeans

k, d = 4, 10

for N in [10**power for power in range(1,8)]:
    data, real_centroids = generate_data(k=k, n=N, d=d, cmin=0, cmax=100)
    start_time = time()
    assignments = KMeans(n_clusters=k, random_state=0, max_iter=1).fit_predict(data)
    elapsed_time = time()-start_time
    print(f"Finished sklearn-kmeans on {N} points in {np.round(elapsed_time,4)} seconds")
    del data, real_centroids
print("Done")

### Precision

While GPU performance is faster than multi-core CPU, it could be faster.
By default, numpy creates arrays in FP64 precision.
This is great for high numerical precision, but unless you're on a \[VAHB\]100 GPU with FP64 cores, performance will be limited.

As shown in the table below ([source](https://www.exxactcorp.com/blog/components/NVIDIA-L40S-GPU-Compared-to-A100-and-H100-Tensor-Core-GPU)), only high end data center cards have FP64 cores.
If you do process FP64 data, the program won't error out, but performance will be sub-optimal because RAPIDS is using emulated FP64 matrix operations on the backend.

|                      | A100 80GB SXM | NVIDIA L40S  | H100 80 GB SXM |
|----------------------|---------------|--------------|----------------|
| GPU Architecture     | NVIDIA Ampere | Ada Lovelace | Hopper         |
| GPU Memory           | 80GB HBM2e    | 48GB GDDR6   | 80GB HBM3      |
| GPU Memory Bandwidth | 2039 GB/s     | 864 GB/s     | 3352 GB/s      |
| L2 Cache             | 40MB          | 96MB         | 50MB           |
| FP64                 | 9.7 TFLOPS    | N/A          | 33.5 TFLOPS    |
| FP32                 | 19.5 TFLOPS   | 91.6 TFLOPS  | 66.9 TFLOPS    |

The easiest way to change this is by casting the data to FP32 or lower.
You've probably done this in PyTorch with BF16 or "mixed-precision" settings, but this is what's happening on the back end.

In [ ]:
# https://docs.rapids.ai/api/cuml/stable/api/#k-means-clustering
from cuml.cluster import KMeans

k, d = 4, 4

for N in [10**power for power in range(1,8)]:
    data, real_centroids = generate_data(k=k, n=N, d=d, cmin=0, cmax=100)
    
    # Cast the data to FP32
    data = data.astype(np.float32)
    
    start_time = time()
    assignments = KMeans(n_clusters=k, random_state=0, max_iter=1, n_init='auto').fit_predict(data)
    elapsed_time = time()-start_time
    print(f"Finished cuml-kmeans on {N} points in {np.round(elapsed_time,4)} seconds")
    del data, real_centroids
print("Done")

In my tests on a T4, I'm seeing a 2x speedup - just from recasting the data!

### Comparing the runtime of each method

While we've been printing out the runtime for each implementation at different data sizes, it's hard to compare the methods.
We can collect all the timing data in python lists and then plot them using [Matplotlib](https://matplotlib.org/).

In [ ]:
# https://docs.rapids.ai/api/cuml/stable/api/#k-means-clustering
from cuml.cluster import KMeans as cuml_KMeans
from sklearn.cluster import KMeans
from kmeans import k_means

k, d = 4, 4

numpy_t = [[], []]
sklearn_t = [[], []]
cuml_t = [[], []]

for N in [10**power for power in range(1,8)]:
    # Use same data for all implementations
    data, real_centroids = generate_data(k=k, n=N, d=d, cmin=0, cmax=100)

    start = time()
    k_means(data, k, max_iterations=1)
    numpy_t[1].append(time()-start)
    numpy_t[0].append(N)

    start = time()
    assignments = KMeans(n_clusters=k, random_state=0, max_iter=1, n_init='auto').fit_predict(data)
    sklearn_t[1].append(time()-start)
    sklearn_t[0].append(N)
    
    # Cast the data to FP32 for cuML
    data = data.astype(np.float32)
    
    start = time()
    assignments = cuml_KMeans(n_clusters=k, random_state=0, max_iter=1, n_init='auto').fit_predict(data)
    cuml_t[1].append(time()-start)
    cuml_t[0].append(N)
    del data, real_centroids
print("Done")

Now that we've collected the data, we can plot it.
Since the runtimes are short and data exponentially increases, we're using a [loglog](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.loglog.html) plot to plot the data on a log scale, so trends are linear.

In [ ]:
plt.figure()
plt.loglog(numpy_t[0], numpy_t[1], label="numpy")
plt.loglog(sklearn_t[0], sklearn_t[1], label="sklearn")
plt.loglog(cuml_t[0], cuml_t[1], label="cuml")
plt.xlabel("# Points")
plt.ylabel("Time (s)")
plt.title("Time to run a single iteration of k-means")
plt.legend()
plt.show()

## K-means Exercises:

### 1. Try increasing the data dimension or count
- Do any methods eventually fail?

### 2. Try modifying `generate_data` to create [cupy arrays](https://docs.cupy.dev/en/stable/user_guide/basic.html)
  - Does this make the cuML method faster?

### 3. Try different levels of [precision](https://docs.cupy.dev/en/stable/overview.html#overview)
  - Is there a speed up on the CPU side as well?
  - Do the computed centroids change?

## cuml.accel: Zero Code Change

Starting in RAPIDS 25.02.01, cuML can now automatically accelerate code that uses Scikit-Learn, UMAP-Learn, and HDBSCAN.
Simply change your invocation to include `-m cuml.accel` and the cuML libraries will be imported instead to run on NVIDIA GPUs and falling back to CPU when necessary.
Give it a try with our kmeans code using [sklearn_kmeans.py](sklearn_kmeans.py):

In [ ]:
!export OMP_NUM_THREADS=16; python sklearn_kmeans.py

# scikit-learn scripts can also be converted directly
!python -m cuml.accel sklearn_kmeans.py

## CuPy: Accelerating NumPy

Similar to RAPIDS, [CuPy](https://cupy.dev/) provides a NumPy-compatible interface for GPU-accelerated computing.
CuPy leverages CUDA for parallel processing, making it ideal for applications requiring large-scale data processing and complex mathematical computations.

Going full-circle, go back and open our [kmeans.py](kmeans.py) file again.
You'll notice that the timing code we've been running is in the `__main__` portion of the script, so it will run with a normal `python kmeans.py` invocation.
We're also only using numpy and built-in python modules.
Since CuPy is meant to replace NumPy, we should be able to just change the import, and see what works...

Make that change and try running the script.

In [ ]:
# After changing the import
!python kmeans.py

If all went well, you should see a good speedup!

### Exercise:

- Look through code of your own to see where CuPy would be useful